# Генерация данных #

### Загрузка пакетов ###

In [35]:
#обработка музыки
import librosa

# загрузка mp3 файлов
import audioread.ffdec  # Use ffmpeg decoder
import ffmpeg

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams["figure.figsize"] = (20,10)

#файлы в директории
from glob import glob

#воспроизведение музыки
from IPython.display import Audio

#сохранение музыки
import soundfile as sf
import json
import csv

In [44]:
#класс генератора данных 
class DataGenerator:
    def __init__(self, audio_path=None):
        """
        Инициализация объекта класса DataGenerator
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            self.path - путь к файлу
            self.y - звуковой временной ряд
            self.t - частота дискретизации y
            self.samples - нарезанные музыкальные фрагменты
            self.number_of_samples - количество нарезанных фрагментов
        """
        
        self.path = audio_path
        self.name = None
        self.y = None
        self.sr = None
        self.samples = None
        self.number_of_samples = None
        self.noised_samples = None
        self.noise_level = None
        
        
    def read_audio(self, audio_path):
        """
        Чтение аудиофайла, лежащего в audio_path
        
        Параметры:
            audio_path (str) - путь к аудиофайлу
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибуты класса:
                y (np.ndarray) - звуковой временной ряд
                sr (int > 0) - частота дискретизации y
                name - название текущего файла
        """
        self.y, self.sr = librosa.load(audioread.ffdec.FFmpegAudioFile(audio_path))
        
        name_with_extension = audio_path.split('\\')[-1]
        self.name = name_with_extension.split('.')[0]
        
        
    def to_samples(self, t = 7.5):
        """
        Нарезает звуковой временной ряд self.y на дорожки длиной t секунд
        
        Параметры:
            t (number or np.ndarray) - длина нарезки аудиофрагментов
            
        Возвращает:
            None
            
        Примечание:
            Инициализирует атрибут класса:
                self.music_fragments (np.ndarray) - массив, содержащий звуковые временные
                    ряды длины t
            
            Нарезанные фрагменты не перекрывают друг друга, за исключением, может быть, последнего,
                который подгоняется под длину t
        """
        self.number_of_samples = librosa.time_to_samples(t, sr = self.sr)
        _music_fragments = []
        
        #нарезка
        for i in range(int(np.ceil(self.y.shape[0] / self.number_of_samples))):
            _music_fragments.append(self.y[i * self.number_of_samples: (i + 1) * self.number_of_samples])
            
        #приведение последнего объекта к 7.5 секундам
        _music_fragments[-1] = np.append(_music_fragments[-1], np.zeros((self.number_of_samples - _music_fragments[-1].shape)))
        
        self.samples = np.array(_music_fragments)
        
    
    def get_samples(self):
        """
        Выгружает нарезанные фрагменты длины t
        
        Параметры:
            None
            
        Возвращает:
            self.samples (np.ndarray) - массив, содержащий звуковые временные ряды длины t
            
        Примечание:
            Использует метод to_samples
        """
        return self.samples
   

    def add_noise(self, noise_level):
        
        """
        Добавляет шум к нарезанным музыкальным фрагментам samples
            в степени noise_level
            
        Параметры:
            samples (np.ndarray) - массив, содержащий звуковые временные ряды
            noise_level (number) - степень шума. от 0 до 1
            
        Возвращает:
            samples + noise (np.ndarray) - нарезанные музыкальные фрагменты с добавленным шумом
            
        Примечание:
        """
        self.noise_level = noise_level
        self.noised_samples = self.samples + np.random.normal(0, noise_level, self.number_of_samples)
        
        
    def save_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                имя-исходного-файла_номер-фрагмента.wav
        """
        for i in range(len(self.samples)):
            sf.write(save_dir + self.name + "_" + str(i) + ".wav", self.samples[i], self.sr)
           
        
    def save_noised_samples(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples с добавленным шумом в степени noise_level
            в директории save_dir
            
        Параметры:
            save_dir (str) - директория, куда сохраняются файлы
            noise_level (number) - степень шума. от 0 до 1
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                noised_степень-шума_имя-исходного-файла_номер-фрагмента.wav
        """
        
        for i in range(len(self.noised_samples)):
            sf.write(save_dir + "noised_" + str(self.noise_level) + "_"+ self.name + "_" 
                     + str(i) + ".wav", self.noised_samples[i], self.sr)
            
            
    def save_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir + self.name + '.json', "w") as file:
            _data = {'init_dict': {'name': self.name, 'sr': self.sr, 'noised': 0, 'noise_level': None}, 
                          'samples': [list(_) for _ in self.samples]}
            file.seek(0)
            json.dump(_data, file)
            
    def save_noised_to_json(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты samples
            в json файле save_dir
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
            Сохраненные файлы называются по следующему принципу:
                [{
                    init_dict:
                        {
                        name: str
                        sr: int > 0
                        noised: bool
                        noise_level: number
                        },
                    samples: list of lists of numbers
                 } , ...]
        """
        with open(save_dir + self.name + '.json', "w") as file:
            _data = {'init_dict': {'name': self.name, 'sr': self.sr, 'noised': 1, "noise_level": self.noise_level}, 
                          'samples': [list(_) for _ in self.noised_samples]}
            file.seek(0)
            json.dump(_data, file)
    
    def save_to_csv(self, save_dir):
        """
        Сохраняет нарезанные музыкальные фрагменты csv файле
            
        Параметры:
            save_dir (str) - json файл, куда сохраняются данные
            
        Возвращает:
            None
            
        Примечание:
        
        """
        with open(save_dir + self.name + '.csv', 'w', newline='') as file:
            spamwriter = csv.writer(file)
            spamwriter.writerow([self.name, self.sr])
            for sample in self.noised_samples:
                spamwriter.writerow(sample)
    
    def generate_noised_data(self, audio_path, save_dir, _type = 'csv'):
        """
        Генерирует зашумлённое аудио по пути audio_path,
        сохраняет в директорию по пути save_dir
        
            
        Параметры:
            audio_path (str) - путь до музыкального файла
            save_dir (str) - директория сохранения
            _type(str) - csv или sample, показывает тип сохранения данных
            
        Возвращает:
            None
            
        Примечание:
       
        """
        self.read_audio(audio_path)
        self.to_samples()
        self.add_noise(0.02)
        if _type == 'sample':
            self.save_noised_samples(save_dir)
        elif _type == 'csv':
            self.save_to_csv(save_dir)

In [45]:
def generate_data(dataGen, audio_path, save_dir, _type = 'csv'):
    musicfiles = glob(audio_path+ "\*.mp3")
    for song in musicfiles:
        dataGen.generate_noised_data(song, save_dir, _type)

In [46]:
ex = DataGenerator()

In [47]:
audio_path = '.\\music'
save_dir = '.\\wav_data\\'
generate_data(ex, audio_path, save_dir, 'csv')

In [ ]:
with open('eggs.csv', 'w', newline='') as csvfile:
    spamwriter = csv.writer(csvfile)
    spamwriter.writerow([0,0,1,1,1,1])